In [1]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [88]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [119]:
lp = StanProblem(bs_model);
D = 10;
initial_θ = rand(D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling  11%|███▌                           |  ETA: 0:00:01
  iterations:                                   895
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.02
  n_steps:                                      31
  is_accept:                                    true
  acceptance_rate:                              0.9763927659190847
  log_density:                                  -7.914204277695983
  hamiltonian_energy:                           16.06550245844311
  hamiltonian_energy_error:                     0.017868046272976557
  max_hamiltonian_energy_error:                 0.09941313307730226
  tree_depth:                                   5
  numerical_error:                              false
  step_size:                                    0.14914249053326567
  nom_step_size:                                0.14914249053326567
  is_adapt:                                     true



















Sampling  40%|████████████▎                  |  ETA: 0:00:00
  iterations:                                   3161
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9064916300019851
  log_density:                                  -5.680410764834427
  hamiltonian_energy:                           12.127811642625804
  hamiltonian_energy_error:                     -0.06584808204247139
  max_hamiltonian_energy_error:                 0.18131248311070536
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false



















Sampling  67%|████████████████████▊          |  ETA: 0:00:00
  iterations:                                   5354
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              1.0
  log_density:                                  -3.125199584845127
  hamiltonian_energy:                           7.977536678133459
  hamiltonian_energy_error:                     -0.13366825911156255
  max_hamiltonian_energy_error:                 -0.13366825911156255
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false



















Sampling  87%|███████████████████████████▏   |  ETA: 0:00:00
  iterations:                                   6994
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9772925684495671
  log_density:                                  -5.848456318382831
  hamiltonian_energy:                           8.064871339961133
  hamiltonian_energy_error:                     0.04319032639595477
  max_hamiltonian_energy_error:                 -0.07950543492963558
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.4052342918202954
  log_density:                                  -8.868373854993063
  hamiltonian_energy:                           19.97580352442992
  hamiltonian_energy_error:                     -0.15188092584388002
  max_hamiltonian_energy_error:                 6.784952120901632
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false
  mass_matrix:                    

┌ Info: Finished 8000 sampling steps for 1 chains in 0.497411802 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.9964929543071683, 0.8576 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.404), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9772347795922866
│   average_acceptance_rate = 0.8720347766900625
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [151]:
    constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
    unc_sample_array = vcat([
            param_unconstrain(bs_model, collect(row[1]))' for row in eachrow(constrained_draws)
        ]...);

MethodError: MethodError: no method matching param_unconstrain(::StanModel, ::Array{Float64, 0})
Closest candidates are:
  param_unconstrain(::StanModel, !Matched::Vector{Float64}) at ~/.julia/packages/BridgeStan/nCPwJ/src/model.jl:422

In [150]:
unc_sample_array

10000×10 Matrix{Float64}:
 10.6803    9.71771     7.77507   …  13.7361     9.33885   0.584418
  6.45384   4.41031     7.0133        3.74315    3.69197   0.767165
 -2.24163   0.761705   -4.74619      -2.25603   -1.01337   0.830766
  2.46002   4.44563     4.31378       3.4816     2.66779   0.535441
  5.62489   3.70945     6.11008       7.07705    4.04478   0.236075
  7.50337  10.2793      4.94622   …  11.9474     7.19249   1.24666
  7.816    -0.746158    6.79006       8.223      6.13427   1.51619
  5.0193    8.18897     9.54053       7.11512    7.30456   1.29114
 10.0454   11.8259      8.17946       3.71629    6.62392   1.35851
  5.18654   7.70174     2.6935        4.82504    6.47553   0.764239
  ⋮                               ⋱                       
  2.78882   0.0901486   1.3044        2.16824    3.87727   1.13078
  5.85363  11.1192     -3.62347       0.246698   6.69739   1.61771
  7.38423   8.7937      4.80441       6.40357    6.40207   1.3757
  2.12388   1.89619     1.47158       1

In [134]:
mean(samples)

10-element Vector{Float64}:
  0.3173551467310577
  0.08433652958871951
 -0.0886958945091761
  0.06037996882064286
 -0.17066047954097616
 -0.06670302452397554
  0.37302740430830394
  0.059786873877665854
  4.384754324471194
  0.806020045873587